# Chapter 10 - 시퀀스 해킹, 해시, 슬라이스

## Vector 버전 #1: Vector2d 호환

In [1]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octects):
        typecode = chr(octects[0])
        memv = memoryview(octects[1:]).cast(typecode)
        return cls(memv)

\_\_repr\_\_()을 구현할 때 ```reprlib.repr(list(self._components))``` 문장으로 components를 간략히 출력할 수도 있었다.<br>
하지만 단순히 list.repr()를 사용하기 위해 self.\_components의 모든 항목을 list에 복사하는 것은 낭비다.<br>
대신 reprlib.repr()을 self.\_componets 배열에 직접 적용하고 나서 \[\] 바깥쪽에 있는 글자들을 잘라냈다.

## 프로토콜과 덕 타이핑

객체지향 프로그래밍에서 프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스다.<br>
예를 들어 파이썬의 시퀀스 프로토콜은 \_\_len\_\_()과 \_\_getitem\_\_() 메서드를 동반할 뿐이다.<br>
표준 시그너처와 의미에 따라 이 메서드들을 구현하는 어떠한 클래스도 시퀀스가 필요한 곳에 사용될 수 있다.<br>
그 클래스의 슈퍼클래스가 무엇인지는 중요하지 않다. **단지 필요한 메서드만 제공**하면 된다.<br>
<br>
프로토콜은 비공식적이며 강제로 적용되는 사항이 아니므로 클래스가 사용되는 특정 환경에 따라 프로토콜의 일부만 구현할 수도 있다.<br>
예를 들어 반복을 지원하려면 \_\_getitem\_\_() 메서드만 구현하면 되며, \_\_len\_\_() 메서드를 구현할 필요는 없다.

## Vector 버전 #2: 슬라이스 가능한 시퀀스

객체 안에 들어있는 시퀀스 속성에 위임하면 시퀀스 프로토콜을 구현하기 위한 \_\_len\_\_()과 \_\_getitem\_\_() 메서드를 다음과 같이 구현할 수 있다.

In [2]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octects):
        typecode = chr(octects[0])
        memv = memoryview(octects[1:]).cast(typecode)
        return cls(memv)
    
    # 추가
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, idx):
        return self._components[idx]

In [3]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [4]:
v1[0], v1[-1]

(3.0, 5.0)

In [5]:
v2 = Vector(range(7))
v2[1:4]

array('d', [1.0, 2.0, 3.0])

## Vector 버전 #3: 동적 속성 접근

속성을 찾지 못하면 인터프리터는 \_\_getattr\_\_() 메서드를 호출한다.<br>
my_obj.x 표현식이 주어지면,

1. my_obj 객체에 x 속성이 있는지 검사
2. 없으면 이 객체의 클래스(my_obj.\_\_class\_\_)에서 검색, 상속 그래프를 따라 계속 올라감
3. 그래도 x 속성을 찾지 못하면 self와 속성명을 문자열로 전달해서 my_obj의 클래스에 정의된 \_\_getattr\_\_() 메서드를 호출

In [8]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octects):
        typecode = chr(octects[0])
        memv = memoryview(octects[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, idx):
        return self._components[idx]
    
    # 추가
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

In [9]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [10]:
v.x

0.0

In [12]:
v.x = 10
v.x

10

In [13]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

파이썬은 해당 이름의 속성을 찾지 못할 때 최후 수단으로 \_\_getattr\_\_() 메서드를 호출한다.<br>
그러나 v.x = 10 문장으로 x 속성에 값을 할당할 때 v 객체에 x 속성이 추가되므로, 더 이상 v.x 값을 가져오기 위해 \_\_getattr\_\_()을 호출하지 않는다.<br>
이를 방지하려면 \_\_setattr\_\_() 메서드를 구현해야 한다.

```python
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
        if name in cls.shortcut_names:
            error = 'readonly attribute {attr_name!r}'
        elif name.islower():
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''
        if error:
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
    super().__setattr__(name, value)
```

## Vector 버전 #4: 해싱 및 더 빠른 ==

다음은 0부터 5까지 정수를 xor로 누적 계산하는 세 가지 방법이다.

In [1]:
n = 0
for i in range(6):
    n ^= i
    
n

1

In [2]:
import functools
functools.reduce(lambda a, b: a ^ b, range(6))

1

In [3]:
import operator
functools.reduce(operator.xor, range(6))

1

In [4]:
import functools
import operator

class Vector:
    # ...
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    # ...

현재 Vector 클래스의 \_\_eq\_\_() 메서드는 수천 개의 요소를 가질 수 있는 Vector 객체의 경우 상당히 비효율적이다.<br>
단지 튜플형의 \_\_eq\_\_() 메서드를 적용하기 위해 피연산자 전체를 복사해서 튜플 두 개를 만든다.<br>
요소가 2개밖에 없는 Vector2d의 경우에는 나쁘지 않지만, 아주 큰 다차원 벡터의 경우에는 얘기가 다르다.<br>
Vector 객체를 다른 Vector 객체나 반복형과 비교할 때는 다음과 같이 구현하는 것이 좋다.

In [ ]:
def __eq__(self, other):
    if len(self) != len(other):
        return False
    
    for a, b in zip(self, other):
        if a != b:
            return False
        
    return True

zip()과 all()을 이용해서 구현할 수도 있다.

In [ ]:
def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))